<a href="https://colab.research.google.com/github/MFarisA/Compu-vision/blob/main/facenet_VGGFACE_OpenFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FACENET**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mtcnn keras-facenet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10368 sha256=a2e0461ff367b57e36342e7c3a3c0bcb452608d759b99d4621378cab3cdc1c46
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os


In [ ]:
detector = MTCNN()

def extract_face(image_path, required_size=(160, 160)):
    img = Image.open(image_path).convert('RGB')
    pixels = np.asarray(img)
    results = detector.detect_faces(pixels)
    if len(results) > 0:
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        face_image = Image.fromarray(face)
        face_image = face_image.resize(required_size)
        face_array = np.asarray(face_image)
        return face_array
    return None


In [ ]:
def load_faces(directory):
    faces = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                img_path = os.path.join(label_dir, filename)
                face = extract_face(img_path)
                if face is not None:
                    faces.append(face)
                    labels.append(label)
    return np.array(faces), np.array(labels)

# Example: Load the train dataset
train_man_dir = '/content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/train'
train_women_dir = '/content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/val'

faces_train_man, labels_train_man = load_faces(train_man_dir)
faces_train_women, labels_train_women = load_faces(train_women_dir)

# Combine man and women data
faces_train = np.concatenate((faces_train_man, faces_train_women))
labels_train = np.concatenate((labels_train_man, labels_train_women))


In [ ]:
import os

print("Contents of train/man:", os.listdir(train_man_dir))
print("Contents of train/women:", os.listdir(train_women_dir))


Contents of train/man: ['aug_7_S124-02-t10_01.jpg', 'aug_10_S146-02-t10_01.jpg', 'aug_7_S194-02-t10_01.jpg', 'aug_5_S093-01-t10_01.jpg', 'aug_4_S194-02-t10_01.jpg', 'aug_6_S045-01-t10_02.jpg', 'aug_0_S198-05-t10_01.jpg', 'aug_4_S198-04-t10_01.jpg', 'aug_5_S198-04-t10_01.jpg', 'aug_1_S198-05-t10_01.jpg', 'aug_7_S146-02-t10_01.jpg', 'aug_9_S058-01-t10_01.jpg', 'aug_8_S194-02-t10_01.jpg', 'aug_6_S198-04-t10_01.jpg', 'aug_5_S194-02-t10_01.jpg', 'aug_5_S058-03-t10_01.jpg', 'aug_9_S194-02-t10_01.jpg', 'aug_4_S146-02-t10_01.jpg', 'aug_0_S045-01-t10_02.jpg', 'aug_5_S045-01-t10_02.jpg', 'aug_8_S124-02-t10_01.jpg', 'aug_2_S198-04-t10_01.jpg', 'aug_3_S194-02-t10_01.jpg', 'S333-02-t10_01.jpg', 'aug_6_S346-02-t10_01.jpg', 'aug_4_S346-02-t10_01.jpg', 'aug_5_S282-03-t10_01.jpg', 'aug_7_S331-02-t10_01.jpg', 'aug_1_S352-03-t10_01.jpg', 'aug_7_S341-01-t10_01.jpg', 'aug_5_S309-03-t10_01.jpg', 'aug_0_S353-01-t10_01.jpg', 'S309-01-t10_01.jpg', 'aug_3_S346-01-t10_01.jpg', 'aug_1_S321-03-t10_02.jpg', 'aug_6_

In [ ]:
def load_faces(directory):
    faces = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                img_path = os.path.join(label_dir, filename)
                # Check if the file is an image
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    face = extract_face(img_path)  # Your face extraction function
                    if face is not None:
                        faces.append(face)
                        labels.append(label)
    return np.array(faces), np.array(labels)


In [ ]:
# Load training data
train_dir = '/content/drive/MyDrive/campuss/Computer-vision/converted_images'
faces_train, labels_train = load_faces(train_dir)
print(f"Loaded {len(faces_train)} faces with labels: {labels_train}")


Loaded 848 faces with labels: ['male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male' 'male'
 'male' 'male' 'male' 'male' 'male' 'male' 'mal

In [ ]:
# Check if labels are empty before encoding
if len(labels_train) > 0:
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels_train)
    print("Encoded Labels:", encoded_labels)

    # Proceed to categorical conversion
    y_train = to_categorical(encoded_labels)
    print("y_train shape:", y_train.shape)
else:
    print("No labels found. Please check the loading function.")


Encoded Labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 

In [ ]:
# After ensuring both classes are present
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels_train)  # This should now have multiple classes
y_train = to_categorical(encoded_labels)

In [ ]:
facenet = FaceNet()

def get_embedding(face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = facenet.embeddings(samples)
    return yhat[0]

embeddings_train = np.array([get_embedding(face) for face in faces_train])


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(embeddings_train, y_train, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(512,)))  # Changed input_shape to (512,)
model.add(Dense(2, activation='softmax'))  # Assuming 2 classes: man and woman

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7685 - loss: 0.5113 - val_accuracy: 0.8706 - val_loss: 0.3887
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9156 - loss: 0.2936 - val_accuracy: 0.8706 - val_loss: 0.4071
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9117 - loss: 0.2998 - val_accuracy: 0.8706 - val_loss: 0.3910
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9104 - loss: 0.3021 - val_accuracy: 0.8706 - val_loss: 0.3943
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9139 - loss: 0.2941 - val_accuracy: 0.8706 - val_loss: 0.3940
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9152 - loss: 0.2908 - val_accuracy: 0.8706 - val_loss: 0.3913
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9067 - loss: 0.3106 - val_accuracy: 0.8706 - val_loss: 0.3954
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9189 - loss: 0.2829 - val_accuracy: 0.8706 - val_loss: 0.

In [ ]:
y_pred = model.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

accuracy = accuracy_score(y_true_labels, y_pred_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Accuracy: 87.06%


# **VGGFACE**

In [ ]:
!pip install tensorflow==2.13.0 --force-reinstall
!pip install keras_vggface
!pip install keras_applications

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 2

In [ ]:
!pip install keras==2.13.1
!pip install keras_applications==1.0.8 --no-deps
!pip install keras_vggface==0.6

Prepraring the Dataset

In [ ]:
import os
import numpy as np
# Import ImageDataGenerator from tensorflow.keras.preprocessing.image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = '/content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/train'
val_dir = '/content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/val'

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Create image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1130 images belonging to 2 classes.
Found 534 images belonging to 2 classes.


Load VGG FACE Model

In [ ]:
!pip install keras_applications

  Using cached Keras_Applications-1.0.8-py3-none-any.whl.metadata (1.7 kB)
Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)


In [ ]:
!pip install keras_vggface --upgrade

In [ ]:
!pip install keras==2.13.1
!pip install tensorflow==2.13.1

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras_vggface.vggface import VGGFace # Import VGGFace from keras_vggface.vggface

# Load VGG-FACE model
base_model = VGGFace(weights='vggface', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # Adjust the number of classes if needed

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

ImportError: cannot import name 'layer_utils' from 'keras.utils' (/usr/local/lib/python3.10/dist-packages/keras/utils/__init__.py)

Compile Model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train Model

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=val_generator
)

Evaluate Model

In [ ]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

Save Model

In [ ]:
model.save('/content/drive/MyDrive/campuss/Computer-vision/data-for-any/model-any/vggface_model.h5')


# **OpenFace**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!apt-get install -y cmake libopenblas-dev liblapack-dev libboost-all-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
liblapack-dev is already the newest version (3.10.0-2ubuntu1).
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
libopenblas-dev is already the newest version (0.3.20+ds-1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 63 not upgraded.


In [8]:
!pip install dlib

In [10]:
!git clone https://github.com/TadasBaltrusaitis/OpenFace.git
%cd OpenFace

Cloning into 'OpenFace'...
remote: Enumerating objects: 21343, done.
remote: Total 21343 (delta 0), reused 0 (delta 0), pack-reused 21343 (from 1)
Receiving objects: 100% (21343/21343), 1.66 GiB | 23.20 MiB/s, done.
Resolving deltas: 100% (11222/11222), done.
Updating files: 100% (3672/3672), done.
/content/OpenFace/OpenFace/build/build/OpenFace


In [13]:
!git clone https://github.com/davisking/dlib.git
%cd dlib
!mkdir build
%cd build
!cmake ..
!cmake --build .
!make install

Cloning into 'dlib'...
remote: Enumerating objects: 51914, done.
remote: Counting objects: 100% (459/459), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 51914 (delta 214), reused 284 (delta 106), pack-reused 51455 (from 1)
Receiving objects: 100% (51914/51914), 48.81 MiB | 11.72 MiB/s, done.
Resolving deltas: 100% (36594/36594), done.
/content/OpenFace/OpenFace/build/build/OpenFace/build/build/dlib
/content/OpenFace/OpenFace/build/build/OpenFace/build/build/dlib/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - do

In [15]:
!export dlib_DIR=/content/dlib/build

In [17]:
%cd /content/OpenFace
!mkdir build
%cd build
!cmake ..
!make

/content/OpenFace
/content/OpenFace/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenBLAS libraries: /usr/lib/x86_64-linux-gnu/libopenblas.so
-- Found OpenBLAS include: /usr/include/x86_64-linux-gnu
OpenBLAS information:
  OpenBLAS_LIBRARIES: /usr/lib/x86_64-linux-gnu/libopenblas.so
  OpenBLAS_INCLUDE: /usr/include/x86_64-linux-gnu
-- Found OpenCV: /usr (found suitable version "4.5.4", minimum required is "4.0") found components: core imgproc calib3d highgui objdetect
OpenCV information:
  OpenCV_INCLUDE_DIRS: /usr/incl

In [22]:
!chmod +x ./build/bin/FeatureExtraction

chmod: cannot access './build/bin/FeatureExtraction': No such file or directory


In [19]:
!mkdir -p /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/features/train/male
!mkdir -p /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/features/train/female
!mkdir -p /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/features/val/male
!mkdir -p /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/features/val/female

In [28]:
!ls /content/OpenFace/build/bin

AU_predictors  FaceLandmarkImg	FaceLandmarkVid  FaceLandmarkVidMulti  FeatureExtraction  model


In [29]:
%cd /content/OpenFace/build/bin

/content/OpenFace/build/bin


In [35]:
%cd /content/OpenFace

/content/OpenFace


In [36]:
!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/patch_experts/cen_patches_0.25_of.dat -P ./model/patch_experts
!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/patch_experts/cen_patches_0.35_of.dat -P ./model/patch_experts
!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/patch_experts/cen_patches_0.50_of.dat -P ./model/patch_experts
!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/patch_experts/cen_patches_0.70_of.dat -P ./model/patch_experts

!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/pdms/In-the-wild_aligned_PDM_68.txt -P ./model/pdms
!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/tris_68.txt -P ./model
!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/main_ceclm_general.txt -P ./model
!wget https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/cen_general.txt -P ./model

--2024-10-29 08:19:50--  https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/patch_experts/cen_patches_0.25_of.dat
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-10-29 08:19:50 ERROR 404: Not Found.

--2024-10-29 08:19:50--  https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/patch_experts/cen_patches_0.35_of.dat
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-10-29 08:19:50 ERROR 404: Not Found.

--2024-10-29 08:19:51--  https://github.com/TadasBaltrusaitis/OpenFace/raw/master/models/patch_experts/cen_patches_0.50_of.dat
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-10-29 

In [37]:
!wget https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_alt.xml -P ./model/face_detection

--2024-10-29 08:20:06--  https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_alt.xml
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./model/face_detection/haarcascade_frontalface_alt.xml’

haarcascade_frontal     [ <=>                ]   2.04M  --.-KB/s    in 0.06s   

2024-10-29 08:20:07 (31.7 MB/s) - ‘./model/face_detection/haarcascade_frontalface_alt.xml’ saved [2140197]



In [39]:
%cd /content/OpenFace
!wget -qO- https://github.com/TadasBaltrusaitis/OpenFace/releases/download/v2.0.6/OpenFace_2.0.6_models.zip | jar x

/content/OpenFace


In [41]:
%ls /content/OpenFace/model/face_detection/

haarcascade_frontalface_alt.xml


In [44]:
!./build/bin/FeatureExtraction -fdir /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/train/male -out_dir /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/features/train/male
!./build/bin/FeatureExtraction -fdir /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/train/female -out_dir /content/drive/MyDrive/campuss/Computer-vision/data-for-any/convert/features/train/female

-mloc /content/OpenFace/model/face_detection/

Could not find the HAAR face detector location
Reading the landmark detector/tracker from: /content/OpenFace/model/face_detection/
^C
